In [9]:
import numpy as np
import pandas as pd
from boruta import BorutaPy
from sklearn.datasets import load_diabetes
from sklearn.metrics import accuracy_score 
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

In [5]:
%cd Z:\PPMI_Data\Excels\NonMotors\Derived250\Male\GeneCpG\UPSIT
df1 = pd.read_csv ("UPSIT3Categ_Methylome_Male.csv")
print(df1.shape)
df1.head(2)

Z:\PPMI_Data\Excels\NonMotors\Derived250\Male\GeneCpG\UPSIT
(307, 5679)


,Sentrix,PATNO,HYS,Gender,APPRDX,UPSIT_3,UPSIT_Total,cg06067658,cg15442538,cg25928446,...,cg05766881,cg01708543,cg10933281,cg04735449,cg15856662,cg11864127,cg14420798,cg08859349,cg01481205,cg19228755
0,200991620021_R01C01,3001,2,1,1,1,25,0.088679,0.890148,0.719238,...,0.748763,0.830052,0.861102,0.855659,0.788271,0.805084,0.729165,0.803183,0.925666,0.849608
1,200973410167_R04C01,3007,2,1,1,2,18,0.128668,0.916448,0.726893,...,0.886279,0.898054,0.896800,0.911732,0.863749,0.893595,0.829444,0.866504,0.917233,0.786606


In [6]:
df = df1.drop(['PATNO', 'Sentrix', 'Gender', 'UPSIT_3','UPSIT_Total'], axis=1)
df.head(2)

,HYS,APPRDX,cg06067658,cg15442538,cg25928446,cg12415575,cg26358225,cg19058526,cg20195763,cg23725394,...,cg05766881,cg01708543,cg10933281,cg04735449,cg15856662,cg11864127,cg14420798,cg08859349,cg01481205,cg19228755
0,2,1,0.088679,0.890148,0.719238,0.066678,0.151532,0.133249,0.479724,0.926745,...,0.748763,0.830052,0.861102,0.855659,0.788271,0.805084,0.729165,0.803183,0.925666,0.849608
1,2,1,0.128668,0.916448,0.726893,0.066768,0.198886,0.141136,0.412058,0.876792,...,0.886279,0.898054,0.896800,0.911732,0.863749,0.893595,0.829444,0.866504,0.917233,0.786606


## HYS = 0 in Patient vs Control (All)

In [22]:
df_HYS = df[(df['APPRDX'] == 2) | ((df['APPRDX'] == 1) & (df['HYS'] == 0))]
print(df_HYS.shape)

X = df_HYS.drop(['HYS','APPRDX'], axis = 1)
y = df_HYS['APPRDX']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .2, random_state = 1) 

rf_all_features = RandomForestClassifier(random_state=1)
rf_all_features.fit(X_train, y_train)

accuracy_score(y_test, rf_all_features.predict(X_test))

rfc = RandomForestClassifier(random_state=1)
boruta_selector = BorutaPy(rfc, n_estimators= 'auto', verbose=1, random_state=1)
boruta_selector.fit(np.array(X_train), np.array(y_train))  

# print("Ranking: ",boruta_selector.ranking_)          
print("No. of significant features: ", boruta_selector.n_features_)

Iteration: 1 / 100
Iteration: 2 / 100
Iteration: 3 / 100
Iteration: 4 / 100
Iteration: 5 / 100
Iteration: 6 / 100
Iteration: 7 / 100
Iteration: 8 / 100
Iteration: 9 / 100
Iteration: 10 / 100
Iteration: 11 / 100
Iteration: 12 / 100
Iteration: 13 / 100
Iteration: 14 / 100
Iteration: 15 / 100
Iteration: 16 / 100
Iteration: 17 / 100
Iteration: 18 / 100
Iteration: 19 / 100
Iteration: 20 / 100
Iteration: 21 / 100
Iteration: 22 / 100
Iteration: 23 / 100
Iteration: 24 / 100
Iteration: 25 / 100
Iteration: 26 / 100
Iteration: 27 / 100
Iteration: 28 / 100
Iteration: 29 / 100
Iteration: 30 / 100
Iteration: 31 / 100
Iteration: 32 / 100
Iteration: 33 / 100
Iteration: 34 / 100
Iteration: 35 / 100
Iteration: 36 / 100
Iteration: 37 / 100
Iteration: 38 / 100
Iteration: 39 / 100
Iteration: 40 / 100
Iteration: 41 / 100
Iteration: 42 / 100
Iteration: 43 / 100
Iteration: 44 / 100
Iteration: 45 / 100
Iteration: 46 / 100
Iteration: 47 / 100
Iteration: 48 / 100
Iteration: 49 / 100


BorutaPy finished running.


In [23]:
selected_rf_features = pd.DataFrame({'CpG':list(X_train.columns),
                                       'Ranking':boruta_selector.ranking_})
selected_rf_features = selected_rf_features.sort_values(by='Ranking')

TopCpG = selected_rf_features.iloc[:1, :]
TopCpG.to_csv ('UPSIT_HYS0_ImpCpGList_Male.csv', index = False)
TopCpG.head(2)

,CpG,Ranking
144,cg27606288,1


## HYS = 1 in Patient vs Control (All)

In [24]:
df_HYS = df[(df['APPRDX'] == 2) | ((df['APPRDX'] == 1) & (df['HYS'] == 1))]
print(df_HYS.shape)

X = df_HYS.drop(['HYS','APPRDX'], axis = 1)
y = df_HYS['APPRDX']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .2, random_state = 1) 

rf_all_features = RandomForestClassifier(random_state=1)
rf_all_features.fit(X_train, y_train)

accuracy_score(y_test, rf_all_features.predict(X_test))

rfc = RandomForestClassifier(random_state=1)
boruta_selector = BorutaPy(rfc, n_estimators= 'auto', verbose=1, random_state=1)
boruta_selector.fit(np.array(X_train), np.array(y_train))  

# print("Ranking: ",boruta_selector.ranking_)          
print("No. of significant features: ", boruta_selector.n_features_)

(137, 5674)
Iteration: 1 / 100
Iteration: 2 / 100
Iteration: 3 / 100
Iteration: 4 / 100
Iteration: 5 / 100
Iteration: 6 / 100
Iteration: 7 / 100
Iteration: 8 / 100


BorutaPy finished running.

Iteration: 	9 / 100
Confirmed: 	0
Tentative: 	0
Rejected: 	5672
No. of significant features:  0


In [25]:
selected_rf_features = pd.DataFrame({'CpG':list(X_train.columns),
                                       'Ranking':boruta_selector.ranking_})
selected_rf_features = selected_rf_features.sort_values(by='Ranking')

TopCpG = selected_rf_features.iloc[:1, :]
TopCpG.to_csv ('UPSIT_HYS1_ImpCpGList_Male.csv', index = False)
TopCpG.head(2)

,CpG,Ranking
5500,cg14576059,2


## HYS = 1 in Patient vs Control (All)

In [26]:
df_HYS = df[(df['APPRDX'] == 2) | ((df['APPRDX'] == 1) & (df['HYS'] == 2))]
print(df_HYS.shape)

X = df_HYS.drop(['HYS','APPRDX'], axis = 1)
y = df_HYS['APPRDX']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .2, random_state = 1) 

rf_all_features = RandomForestClassifier(random_state=1)
rf_all_features.fit(X_train, y_train)

accuracy_score(y_test, rf_all_features.predict(X_test))

rfc = RandomForestClassifier(random_state=1)
boruta_selector = BorutaPy(rfc, n_estimators= 'auto', verbose=1, random_state=1)
boruta_selector.fit(np.array(X_train), np.array(y_train))  

# print("Ranking: ",boruta_selector.ranking_)          
print("No. of significant features: ", boruta_selector.n_features_)

(200, 5674)
Iteration: 1 / 100
Iteration: 2 / 100
Iteration: 3 / 100
Iteration: 4 / 100
Iteration: 5 / 100
Iteration: 6 / 100
Iteration: 7 / 100
Iteration: 8 / 100


BorutaPy finished running.

Iteration: 	9 / 100
Confirmed: 	0
Tentative: 	0
Rejected: 	5672
No. of significant features:  0
